📌 week16 과제는 **15주차의 Word Vector, Word Embedding, ULMfit, BERT**으로 구성되어 있습니다. 트랜스포머는 다음 주에 깊게 다루기 때문에 이번 주차에는 넣지 않았습니다.

📌 위키독스의 딥러닝을 이용한 자연어 처리 입문 교재 실습, 관련 블로그 등의 문서 자료로 구성되어 있는 과제입니다. 

📌 안내된 링크에 맞추어 **직접 코드를 따라 치면서 (필사)** 해당 nlp task 의 기본적인 라이브러리와 메서드를 숙지해보시면 좋을 것 같습니다😊 필수라고 체크한 부분은 과제에 반드시 포함시켜주시고, 선택으로 체크한 부분은 자율적으로 스터디 하시면 됩니다.

📌 궁금한 사항은 깃허브 이슈나, 카톡방, 세션 발표 시작 이전 시간 등을 활용하여 자유롭게 공유해주세요!

🥰 **이하 예제를 실습하시면 됩니다.**

1부터 3까지는 필수 과제입니다.

#### 🔹 1-(1) Pre-Trained Word Vector

* 아래 아티클의 **Case Study: Learning Embeddings from Scratch vs. Pretrained Word Embeddings** 부분을 실습하세요


📌 [An Essential Guide to Pretrained Word Embeddings for NLP Practitioners](https://www.analyticsvidhya.com/blog/2020/03/pretrained-word-embeddings-nlp/)







In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#importing libraries
import pandas as pd
import numpy as np

#reading csv files
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Train.csv')
valid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Valid.csv')             

#train_test split
x_tr, y_tr = train['text'].values, train['label'].values
x_val, y_val = valid['text'].values, valid['label'].values

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(x_tr))

#converting text into integer sequences
x_tr_seq  = tokenizer.texts_to_sequences(x_tr) 
x_val_seq = tokenizer.texts_to_sequences(x_val)

#padding to prepare sequences of same length
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=100)
x_val_seq = pad_sequences(x_val_seq, maxlen=100)

In [4]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

112204


In [5]:
#deep learning library
from keras.models import *
from keras.layers import *
from keras.callbacks import *

model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,input_length=100,trainable=True)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 300)          33661200  
                                                                 
 lstm (LSTM)                 (None, 100, 128)          219648    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 33,889,169
Trainable params: 33,889,169
Non-trainable params: 0
____________________________________________

In [6]:
history = model.fit(np.array(x_tr_seq),np.array(y_tr),batch_size=128,epochs=10,validation_data=(np.array(x_val_seq),np.array(y_val)),verbose=1,callbacks=[es,mc])

Epoch 1/10
313/313 [==============================] - ETA: 0s - loss: 0.4060 - acc: 0.8078
Epoch 1: val_acc improved from -inf to 0.86020, saving model to best_model.h5
313/313 [==============================] - 17s 36ms/step - loss: 0.4060 - acc: 0.8078 - val_loss: 0.3162 - val_acc: 0.8602
Epoch 2/10
313/313 [==============================] - ETA: 0s - loss: 0.2075 - acc: 0.9185
Epoch 2: val_acc improved from 0.86020 to 0.86400, saving model to best_model.h5
313/313 [==============================] - 11s 34ms/step - loss: 0.2075 - acc: 0.9185 - val_loss: 0.3419 - val_acc: 0.8640
Epoch 3/10
313/313 [==============================] - ETA: 0s - loss: 0.1053 - acc: 0.9624
Epoch 3: val_acc did not improve from 0.86400
313/313 [==============================] - 10s 31ms/step - loss: 0.1053 - acc: 0.9624 - val_loss: 0.3969 - val_acc: 0.8560
Epoch 4/10
313/313 [==============================] - ETA: 0s - loss: 0.0482 - acc: 0.9849
Epoch 4: val_acc did not improve from 0.86400
313/313 [=======

In [7]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq,y_val, batch_size=128)
print(val_acc)

40/40 [==============================] - 1s 8ms/step - loss: 0.3419 - acc: 0.8640
0.8640000224113464


**GloVe pretrained word embeddings**

In [8]:
import numpy as np

# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/glove.6B.300d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [9]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((size_of_vocabulary, 300))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [10]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,weights=[embedding_matrix],input_length=100,trainable=False)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 300)          33661200  
                                                                 
 lstm_1 (LSTM)               (None, 100, 128)          219648    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 33,889,169
Trainable params: 227,969
Non-trainable params: 33,661,200
____________________________________

In [11]:
history = model.fit(np.array(x_tr_seq),np.array(y_tr),batch_size=128,epochs=10,validation_data=(np.array(x_val_seq),np.array(y_val)),verbose=1,callbacks=[es,mc])

Epoch 1/10
313/313 [==============================] - ETA: 0s - loss: 0.4526 - acc: 0.7805
Epoch 1: val_acc improved from -inf to 0.84520, saving model to best_model.h5
313/313 [==============================] - 10s 22ms/step - loss: 0.4526 - acc: 0.7805 - val_loss: 0.3539 - val_acc: 0.8452
Epoch 2/10
309/313 [============================>.] - ETA: 0s - loss: 0.3448 - acc: 0.8464
Epoch 2: val_acc improved from 0.84520 to 0.86360, saving model to best_model.h5
313/313 [==============================] - 5s 17ms/step - loss: 0.3448 - acc: 0.8461 - val_loss: 0.3156 - val_acc: 0.8636
Epoch 3/10
313/313 [==============================] - ETA: 0s - loss: 0.3006 - acc: 0.8699
Epoch 3: val_acc improved from 0.86360 to 0.86720, saving model to best_model.h5
313/313 [==============================] - 4s 13ms/step - loss: 0.3006 - acc: 0.8699 - val_loss: 0.3005 - val_acc: 0.8672
Epoch 4/10
311/313 [============================>.] - ETA: 0s - loss: 0.2654 - acc: 0.8882
Epoch 4: val_acc improved fro

In [12]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq,y_val, batch_size=128)
print(val_acc)

40/40 [==============================] - 1s 5ms/step - loss: 0.2925 - acc: 0.8788
0.8787999749183655


#### 🔹 1-(2) Word Embeddings

* Keras를 이용해 Word Embeddings를 실습해 봅니다. 아래 페이지를 필사해 주시면 됩니다.

📌 [Using pre-trained word embeddings](https://keras.io/examples/nlp/pretrained_word_embeddings/)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

In [3]:
import os
import pathlib

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of directories: 20
Directory names: ['talk.politics.guns', 'alt.atheism', 'rec.motorcycles', 'comp.os.ms-windows.misc', 'rec.autos', 'soc.religion.christian', 'comp.sys.mac.hardware', 'comp.sys.ibm.pc.hardware', 'rec.sport.hockey', 'sci.crypt', 'talk.religion.misc', 'comp.graphics', 'misc.forsale', 'comp.windows.x', 'rec.sport.baseball', 'talk.politics.misc', 'talk.politics.mideast', 'sci.med', 'sci.space', 'sci.electronics']
Number of files in comp.graphics: 1000
Some example filenames: ['38590', '38316', '38277', '38980', '38691']


In [4]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

In [5]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [6]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [7]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [20]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-06-26 15:03:57--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-26 15:03:57--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-26 15:03:57--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [8]:
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "/content/glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [9]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17923 words (2077 misses)


In [10]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [11]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000200   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         64128     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 128)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, None, 128)        0         
 1D)                                                         

In [12]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [13]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 23s 15ms/step - loss: 2.6667 - acc: 0.1395 - val_loss: 2.0561 - val_acc: 0.2898
Epoch 2/20
125/125 [==============================] - 1s 9ms/step - loss: 1.9300 - acc: 0.3295 - val_loss: 1.5664 - val_acc: 0.4566
Epoch 3/20
125/125 [==============================] - 1s 9ms/step - loss: 1.5204 - acc: 0.4794 - val_loss: 1.3252 - val_acc: 0.5519
Epoch 4/20
125/125 [==============================] - 1s 9ms/step - loss: 1.2748 - acc: 0.5608 - val_loss: 1.1809 - val_acc: 0.5959
Epoch 5/20
125/125 [==============================] - 1s 9ms/step - loss: 1.1114 - acc: 0.6201 - val_loss: 1.3891 - val_acc: 0.5479
Epoch 6/20
125/125 [==============================] - 1s 9ms/step - loss: 0.9579 - acc: 0.6678 - val_loss: 1.2448 - val_acc: 0.5926
Epoch 7/20
125/125 [==============================] - 1s 9ms/step - loss: 0.8444 - acc: 0.7069 - val_loss: 1.0421 - val_acc: 0.6569
Epoch 8/20
125/125 [==============================] - 1s 9ms/step - loss: 

In [14]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]

'comp.graphics'

#### 🔹 2. ELMo Embedding

* 아래는 ELMo Embedding의 작동을 볼 수 있는 case study 코드입니다. 필사하며 ELMo Embedding을 이해해 보세요!


📌 [Elmo Embeddings : A use case study with code](https://medium.com/@errabia.oussama/elmo-embeddings-a-use-case-study-with-code-part-1-e2e5eb7df85c)

In [18]:
pip install allennlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 719 kB 7.0 MB/s 
     |████████████████████████████████| 1.2 MB 55.9 MB/s 
     |████████████████████████████████| 4.0 MB 58.3 MB/s 
     |████████████████████████████████| 125 kB 74.7 MB/s 
     |████████████████████████████████| 1.8 MB 35.1 MB/s 
     |████████████████████████████████| 248 kB 77.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.0 MB 53.5 MB/s 
     |████████████████████████████████| 592 kB 75.4 MB/s 
     |████████████████████████████████| 132 kB 73.9 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
     |████████████████████████████████| 232 kB 79.0 MB/s 
     |████████████████████████████████| 8.9 MB 52.8 MB/s 
     |████████████████████████████████|

In [1]:
import pandas as pd
from sklearn.manifold import TSNE
from allennlp.modules.elmo import Elmo,batch_to_ids
import spacy
import seaborn as sns
import re
import gc

# 여기서 자꾸 오류가 나서 Elmo는 실행하지 못했습니다ㅠㅠ

ModuleNotFoundError: ignored

In [ ]:
warnings.simplefilter('ignore')


# -------Load train data --------

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv', nrows=10000)

input_col = 'comment_text'
target_col = 'target'
CLEAN = True

In [ ]:
misspell_dict = {"aren't": "are not", "can't": "cannot", "couldn't": "could not",
                 "didn't": "did not", "doesn't": "does not", "don't": "do not",
                 "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                 "he'd": "he would", "he'll": "he will", "he's": "he is",
                 "i'd": "I had", "i'll": "I will", "i'm": "I am", "isn't": "is not",
                 "it's": "it is", "it'll": "it will", "i've": "I have", "let's": "let us",
                 "mightn't": "might not", "mustn't": "must not", "shan't": "shall not",
                 "she'd": "she would", "she'll": "she will", "she's": "she is",
                 "shouldn't": "should not", "that's": "that is", "there's": "there is",
                 "they'd": "they would", "they'll": "they will", "they're": "they are",
                 "they've": "they have", "we'd": "we would", "we're": "we are",
                 "weren't": "were not", "we've": "we have", "what'll": "what will",
                 "what're": "what are", "what's": "what is", "what've": "what have",
                 "where's": "where is", "who'd": "who would", "who'll": "who will",
                 "who're": "who are", "who's": "who is", "who've": "who have",
                 "won't": "will not", "wouldn't": "would not", "you'd": "you would",
                 "you'll": "you will", "you're": "you are", "you've": "you have",
                 "'re": " are", "wasn't": "was not", "we'll": " will", "tryin'": "trying"}
def _get_misspell(misspell_dict):
    misspell_re = re.compile('(%s)' % '|'.join(misspell_dict.keys()))
    return misspell_dict, misspell_re


def replace_typical_misspell(text):
    misspellings, misspellings_re = _get_misspell(misspell_dict)

    def replace(match):
        return misspellings[match.group(0)]

    return misspellings_re.sub(replace, text)


# deal with obvuous misspeled word:

train[input_col] = train[input_col].apply(lambda x : str(x).lower())
train[input_col] = train[input_col].apply(replace_typical_misspell)


def clean_text(sentence):
    sentence = str(sentence).lower()
    sentence = re.sub("[^\w']",' ',sentence)
    sentence = re.sub(' +',' ',sentence)
    return sentence

spell_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)
words = spell_model.index2word
w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
WORDS = w_rank
# Use fast text as vocabulary
def words(text): return re.findall(r'\w+', text.lower())
def P(word):
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)
def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)
def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or [word])
def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)
def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def singlify(word):
    return "".join([letter for i,letter in enumerate(word) if i == 0 or letter != word[i-1]])



if not CLEAN:
    class_toxic    = train[train.target>=0.5].comment_text.values.tolist()
    class_positive = train[train.target <0.5].comment_text.values.tolist()[:1000]
else:
    class_toxic = list(map(clean_text,train[train.target >= 0.5].comment_text.values.tolist()))
    class_positive = list(map(clean_text,train[train.target < 0.5].comment_text.values.tolist()[:1000]))

In [ ]:
word_dict= {}
word_index=1
lemma_dict ={}
word_sequences_toxic= []
word_sequences_positive= []

toxic_embed   = []
positive_embed= []

nlp = spacy.load('en_core_web_lg',disable = ['parser','ner','tagger'])
docs = nlp.pipe(class_toxic,n_threads = 6)
for doc in docs:
    word_seq= []
    for token in doc:
        word_seq.append(token.text)
        #if token.text not in word_dict:
        #    word_dict[token.text] = word_index
        #    word_index+=1
        #    lemma_dict[token.text] = token.lemma_
        #    word_seq.append(token.text)
    word_sequences_toxic.append(word_seq)

del spell_model,words,w_rank
gc.collect()


docs = nlp.pipe(class_positive,n_threads = 6)
for doc in docs:
    word_seq= []
    for token in doc:
        word_seq.append(correction(token.text))
        #if token.text not in word_dict:
        #    word_dict[token.text] = word_index
        #    word_index+=1
        #    lemma_dict[token.text] = token.lemma_
        #    word_seq.append(token.text)
    word_sequences_positive.append(word_seq)


del nlp,docs

In [ ]:
options_file = "elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo = Elmo(options_file,weight_file,1,dropout=0)
def elmo_embe_avg(seq_text, top_to_use):
    avg_embeds = []
    for i in range(10):
        if (i*100)>len(seq_text):
            break
        embds = elmo(batch_to_ids(seq_text[i*100:(i+1)*100]))
        for embd, mask in zip(embds['elmo_representations'][0], embds['mask']):
            avg_embeds.append( (torch.sum(embd, dim=0) / torch.sum(mask)).detach().numpy())
    return avg_embeds

class_toxic_embeddings = pd.DataFrame(elmo_embe_avg(word_sequences_toxic,500))
class_toxic_embeddings['target'] = 1
class_positive_embeddings = pd.DataFrame(elmo_embe_avg(word_sequences_positive,500))
class_positive_embeddings['target'] =0
data_all = pd.concat((class_toxic_embeddings,class_positive_embeddings))
data_all.reset_index(inplace=True,drop=True)

In [ ]:
tsne = TSNE(random_state=1991,n_iter=1500,metric='cosine',n_components=2)
tsne.fit(data_all.drop('target',axis=1))

embd_tr = tsne.fit_transform(data_all.drop('target',axis=1) )
data_all['ts_x_axis'] = embd_tr[:,0]
data_all['ts_y_axis'] = embd_tr[:,1]

In [ ]:
sns.scatterplot('ts_x_axis','ts_y_axis',hue='target',data=data_all)

#### 🔹 3. BERT

* 아래 두 예제로 BERT의 작동을 이해해 보세요!

📌 [한국어 BERT의 마스크드 언어 모델(Masked Language Model) 실습](https://wikidocs.net/152922)

📌  [한국어 BERT의 다음 문장 예측(Next Sentence Prediction)](https://wikidocs.net/156774)

--- 

참고: [버트(Bidirectional Encoder Representations from Transformers, BERT)](https://wikidocs.net/115055)

**마스크드 언어 모델**

In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.1 MB/s 
     |████████████████████████████████| 6.6 MB 42.5 MB/s 
     |████████████████████████████████| 101 kB 14.2 MB/s 
     |████████████████████████████████| 596 kB 66.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [6]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')

In [8]:
print(inputs['input_ids'])

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [9]:
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


In [10]:
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


In [11]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [12]:
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.8963507413864136,
  'sequence': '축구는 정말 재미있는 스포츠 다.',
  'token': 4559,
  'token_str': '스포츠'},
 {'score': 0.025957655161619186,
  'sequence': '축구는 정말 재미있는 거 다.',
  'token': 568,
  'token_str': '거'},
 {'score': 0.010033924132585526,
  'sequence': '축구는 정말 재미있는 경기 다.',
  'token': 3682,
  'token_str': '경기'},
 {'score': 0.007924371398985386,
  'sequence': '축구는 정말 재미있는 축구 다.',
  'token': 4713,
  'token_str': '축구'},
 {'score': 0.007844211533665657,
  'sequence': '축구는 정말 재미있는 놀이 다.',
  'token': 5845,
  'token_str': '놀이'}]

In [13]:
pip('어벤져스는 정말 재미있는 [MASK]다.')

[{'score': 0.838240921497345,
  'sequence': '어벤져스는 정말 재미있는 영화 다.',
  'token': 3771,
  'token_str': '영화'},
 {'score': 0.028275690972805023,
  'sequence': '어벤져스는 정말 재미있는 거 다.',
  'token': 568,
  'token_str': '거'},
 {'score': 0.01718943566083908,
  'sequence': '어벤져스는 정말 재미있는 드라마 다.',
  'token': 4665,
  'token_str': '드라마'},
 {'score': 0.01498968992382288,
  'sequence': '어벤져스는 정말 재미있는 이야기 다.',
  'token': 3758,
  'token_str': '이야기'},
 {'score': 0.009382661432027817,
  'sequence': '어벤져스는 정말 재미있는 장소 다.',
  'token': 4938,
  'token_str': '장소'}]

In [14]:
pip('나는 오늘 아침에 [MASK]에 출근을 했다.')

[{'score': 0.08012578636407852,
  'sequence': '나는 오늘 아침에 회사 에 출근을 했다.',
  'token': 3769,
  'token_str': '회사'},
 {'score': 0.06124059483408928,
  'sequence': '나는 오늘 아침에 에 출근을 했다.',
  'token': 1,
  'token_str': '[UNK]'},
 {'score': 0.01748664863407612,
  'sequence': '나는 오늘 아침에 공장 에 출근을 했다.',
  'token': 4345,
  'token_str': '공장'},
 {'score': 0.016131777316331863,
  'sequence': '나는 오늘 아침에 사무실 에 출근을 했다.',
  'token': 5841,
  'token_str': '사무실'},
 {'score': 0.015360759571194649,
  'sequence': '나는 오늘 아침에 서울 에 출근을 했다.',
  'token': 3671,
  'token_str': '서울'}]

**다음 문장 예측**

In [15]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [16]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [17]:
# 이어지는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


In [18]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]


#### 🔹 선택과제 


📌 [구글 BERT의 마스크드 언어 모델(Masked Language Model) 실습](https://wikidocs.net/153992)

📌 [구글 BERT의 다음 문장 예측(Next Sentence Prediction)](https://wikidocs.net/156767)